### Initialization

In [1]:
# install packages
%%capture
!pip install ultralytics
!pip install roboflow
!pip install supervision

In [2]:
from ultralytics import YOLO
from roboflow import Roboflow
from google.colab import userdata
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Detection Model: Object Detection + Tracking


#### YOLO Object Detection (Player, Ball)

In [ ]:
finetuned = False

In [9]:
if finetuned:
    # init finetuned model
    model = YOLO('runs/detect/train3/weights/best.pt') # change this path accordingly
else:
    # init yolov8 model
    model = YOLO('yolov8x')

In [11]:
# predict detections from video
results = model.predict('sample_vod.mp4', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/750) /content/sample_vod.mp4: 384x640 1 ball, 1 goalkeeper, 19 players, 2 referees, 87.8ms
video 1/1 (frame 2/750) /content/sample_vod.mp4: 384x640 1 ball, 1 goalkeeper, 19 players, 2 referees, 67.5ms
video 1/1 (frame 3/750) /content/sample_vod.mp4: 384x640 1 goalkeeper, 20 players, 2 referees, 67.5ms
video 1/1 (frame 4/750) /content/sample_vod.mp4: 384x640 1 ball, 1 goalkeeper, 21 players, 2 referees, 54.7ms
video 1/1 (frame 5/750) 

In [ ]:
print(results[0].boxes[0])
"""
detection result format
    cls: class_id ({0: person, 1: bicycle, ..., 32: sports ball})
    conf: confidence
    data: TBD
    is_track: TBD
    xywh:  x,y coordinates of the bounding box CENTER, and its width and height
"""

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.], device='cuda:0')
conf: tensor([0.9143], device='cuda:0')
data: tensor([[1.3691e+03, 8.1626e+02, 1.4437e+03, 9.0344e+02, 9.1433e-01, 2.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[1406.3904,  859.8503,   74.5415,   87.1750]], device='cuda:0')
xywhn: tensor([[0.7325, 0.7962, 0.0388, 0.0807]], device='cuda:0')
xyxy: tensor([[1369.1196,  816.2628, 1443.6611,  903.4378]], device='cuda:0')
xyxyn: tensor([[0.7131, 0.7558, 0.7519, 0.8365]], device='cuda:0')


'\ndetection result format\n    cls: class_id ({0: person, 1: bicycle, ..., 32: sports ball})\n    conf: confidence\n    data: TBD\n    is_track: TBD\n    xywh:  x,y coordinates of the bounding box CENTER, and its width and height\n'

#### Model Finetuning

In [3]:
rf_key = userdata.get('ROBOFLOW_API_KEY')

rf = Roboflow(api_key=rf_key)
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(1)
dataset = version.download("yolov5")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to football-players-detection-1 in yolov5pytorch:: 100%|██████████| 1338/1338 [00:01<00:00, 683.95it/s] 


In [ ]:
# in data.yaml: rename train and valid datasets to ../train and ../valid
!sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

In [6]:
!yolo task=detect mode=train model=yolov5xu.pt data={dataset.location}/data.yaml batch = 16 epochs=10 imgsz=640

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov5xu.pt, data=/content/football-players-detection-1/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

In [7]:
finetuned = True